<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2025/images/logo.png?raw=true" alt="2022年度ゲノム情報解析入門" height="100px" align="middle">

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>

大規模なゲノムデータを用いた解析: Genomic Prediction その1
---

　今回から、大規模なゲノムデータセットを用いた解析の中でGenomicPredictionを勉強・体験していきます。

大規模ゲノムデータ解析シリーズ:
- GWAS: Genome wide association study(GWAS)
- **Genomic Prediction入門 : Genomic Predictionとは**, 機械学習の基本, 統計モデル, 育種への応用...等々

※今回のテキストはPythonになります。

# 今回の勉強内容
　このテキストでは、そのGenomic Prediction入門の第1回として、以下を勉強します。

1. Genomic Predictionとは？
  1. Genomic Selectionの登場
  1. GWAS等との違い
  1. Genomic Predictionの概要
  1. Genomic Predictionの応用
1. Genomic Predictionの解析手順・必要な知識等
  1. データの準備
  1. 予測モデルの構築
  1. 構築したモデルの応用
1. Genomic Predictionを体験してみる
  1. 理想的な遺伝子型を探してみる
  1. 最適な交配組み合わせを探してみる

---

　一度次のコードセルを実行して、実習に必要なライブラリのインストールや、プログラム、サンプルファイルのダウンロードしてください。

In [ ]:
################################
##  以下の実習の前に一度実行してください。##
################################

## プログラム・サンプルファイルのダウンロード
!wget -q -O 'GP_linear_data.csv' https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2023/dataset/GP_linear_data.csv
!wget -q -O 'GP_linear_data2.csv' https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2023/dataset/GP_linear_data2.csv
!wget -q -O 'GP_sample_genotype.csv' https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2023/dataset/GP_sample_genotype.csv
!wget -q -O 'GP_sample_phenotype.csv' https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2023/dataset/GP_sample_phenotype.csv
!wget -q -O genomic_prediction.py https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2023/scripts/genomic_prediction_1.py?raw=true
## 確認
!ls | grep -e 'GP_linear_data.csv' -e 'GP_linear_data2.csv' -e 'GP_sample_genotype.csv' -e 'GP_sample_phenotype.csv' -e 'genomic_prediction.py'

# 1. Genomic Predictionとは？

これまで扱ってきたQTL-seqやGWASという手法は、形質に影響を与えている原因遺伝子を探るためにQTLを探索する手法です。

**Genomic Prediction(ゲノミック予測)**モデルは、**Genomic Selection(ゲノミック選抜)**と呼ばれる選抜法で重要となるモデルになります。

まずはこの**Genomic Selection**について、簡単に説明していこうと思います。

## 1-1. Genomic Selectionの登場

生物の形質が変化する要因の一つとして**Selection(選抜)**があることは御存知かと思います。

選抜の中には、よく知られている自然選択のほかに、人為的な選抜(artificial selection)が存在し、『種の起源』の一節でも触れられています。

人為的な選抜は植物の栽培種や動物の家畜化に大きな影響を与えてきました。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/artificial_selection.png?raw=true" alt="artificial_selection" height="400px" align="middle">

この様な選抜は、育種の世界でも行われてきており、栽培品種の中でも良い形質を示す系統を掛け合わせてより良い系統の作出が行われてきていました。

今でも育種家や種苗会社では形質を基にした選抜や交配が行われており、表現型値を元に選抜していくので、**Phenotypic Selection**と呼ばれたりします。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/phenotypic_selection.png?raw=true" alt="phenotypic_selection" height="250px" align="middle">

ただしPhenotypic Selectionには問題点もあります。

形質の情報だけでは最適な組み合わせを正確に選抜できない可能性があったり、畜産の世界では形質値を評価出来るまで生育するのに数年間かかるので選抜までに大変時間がかかります。

また、形質値には環境の影響が出てしまうので、偶然形質値が高く(低く)なってしまう場合もあったりします。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/phenotypic_selection_limit.png?raw=true" alt="phenotypic_selection_limit" height="600px" align="middle">

そこで、ゲノム情報に基づいた選抜が考案されるようになります。

有名な手法の一つに**Marker-Assisted Selection(MAS)(マーカー選抜)**というものがあります。

前回までに扱っていたQTL-seqやGWASによって、形質に影響を与えている遺伝子座(QTL)が分かるようになりました。

形質への効果が分かっているQTL等の遺伝子座を基に選抜する手法の一つがMarker-Assisted Selection(MAS)になります。

目的のQTLを集積させていくことで、より良い系統を作出していく形になります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2025/images/marker_selection.png?raw=true" alt="marker_selection" height="300px" align="middle">

しかし、MASでは１つや２つのQTLを集積させる分には問題ありませんが、たくさんの遺伝子が形質に関与している場合、QTLの同定や集積が難しくなります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/marker_selection_limit.png?raw=true" alt="marker_selection_limit" height="300px" align="middle">

そこで、限られたマーカー(QTL)の情報だけではなく、ゲノム領域全体の情報を用いて選抜を行う方法が開発されました。

それが**Genomic Selection (ゲノミック選抜)**です。

細かい所は後ほど説明していきますが、

ゲノム情報全体から**ゲノム育種価**と呼ばれる、ゲノム全体が形質に与える効果を推定し、選抜を行う方法になります。

ゲノム情報から形質を予測することが出来るので、形質評価が出来るところまで成長するのを待つ必要もなくなります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/genomic_selection.png?raw=true" alt="genomic_selection" height="400px" align="middle">

この時、ゲノム情報からゲノム育種価を予測するための予測モデルが**Genomic Predictionモデル**になります。

## 1-2. GWAS等との違い

これまで扱ってきたQTL-seqやGWASという手法は、形質に影響を与えている原因遺伝子を探るためにQTLを探索する手法でした。

具体的には、異なる系統を交配することでゲノムがシャッフルされた分離集団を作成したり、遺伝子型が異なる系統を集めて、遺伝的背景が異なる集団を作成し、シーケンスデータと形質値データを集めて解析することで、形質値に関与していそうな遺伝領域を探る手法でした。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/gene_mapping.png?raw=true" alt="gene_mapping" height="280px" align="middle">

<br><br>

これらの手法は、**「形質に関与している原因遺伝子の特定」**というのが大きな目的でした。

遺伝子を同定してその機能を調べていくという研究において非常に重要なアプローチの一つになります。

しかし、実際に育種を進めていく上で、
* 形質に影響を与える遺伝子がそれぞれどのくらい効果を持っているのか？
* その遺伝子に変異が入ったり、別の系統と交配し遺伝子型が変わることで、最終的にどのような形質値になるのか？
* 多数の遺伝子に支配されており、効果が分散されている場合、それぞれの遺伝子の効果をどう捉えるか？

といった点には対応できません。<br><br>

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2025/images/GP_gene_effect.png?raw=true" alt="GP_gene_effect" height="250px" align="middle">

<br>この様な問題に対してもGenomic Prediction(ゲノミック予測)であれば対応が可能になります。

Genomic Predictionは「ゲノム全体の遺伝子型情報から、その生物がどんな形質値になるのか」を予測するアプローチになります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/GP_image.png?raw=true" alt="GP_image" height="200px" align="middle">


そのため、GWAS等で原因遺伝子が検出されない形質に対しても、

ゲノム全体がどれだけ形質に影響を与えているかを基準にして選抜育種等を実行することが出来ます。

ものすごくざっくりと言うと、

* GWASやQTL解析は、形質に関連している遺伝子座(QTL)を検出する方法
* Genomic Preidctionは、ゲノム全体(全QTL)の形質に対する影響力を予測する方法

ということになります。

## 1-3. Genomic Predictionの概要
　Genomic Predictionは「ゲノム全体の遺伝子型情報から、その生物がどのくらいの形質値になるのか」を予測するアプローチということでした。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/GP_image.png?raw=true" alt="GP_image" height="200px" align="middle">

　上図の様に、作物の遺伝子型を調べて、その情報から作物がどのように育っていくのか予測する様なイメージです。

　とても簡単に言うと

$$葉の長さ = 遺伝子Aの効果×12.4 + 遺伝子Bの効果×7.6 - 遺伝子Cの効果×8.4+...$$

みたいな感じですね。こんなモデルが出来れば、どの遺伝子を持っているか調べるだけで葉の長さが計算できます。<br><br>

　この様な予測モデルを構築するためには、遺伝子型と形質値の関係性をうまく捉える必要があります。

遺伝子型と形質値の関係性を捉えるためには、十分な量の遺伝子型と形質の組み合わせの情報が必要になり、

これまで扱ってきたQTL-seqやGWASで必要なデータセットよりも更に大規模なデータセットが必要になってきます。

　大規模な交配集団等の準備とその遺伝子型の調査(Genotyping)、及び形質値の測定(Phenotyping)が必要になり、高コストな研究の１つです。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/GP_data.png?raw=true" alt="GP_data" height="400px" align="middle">

　近年はシーケンスの技術が向上しコストが低下してきたことで様々な作物で予測モデルを構築する試みが可能になってきました。

また、機械学習の技術等、情報科学が発展してきたこともあり、様々な形質に対応した予測モデルの開発が可能となってきています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/GP_moore.png?raw=true" alt="GP_moore" height="250px" align="middle">

　まとめると、Genomic Prediction(ゲノミック予測)モデルは、統計学的な知識・技術を活用して、

遺伝子型と形質値の大規模なデータから、ゲノムと形質の関係性を上手く説明する様な予測モデルを構築したものになります。

## 1-4. Genomic Predictionの応用

　それでは、良いGenomic Predictionモデルの構築に成功し、「ある遺伝子型がどのくらいの形質値になるのか」を予測することが出来るとどのようなことに活用できるのでしょうか。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_image.png?raw=true" alt="GP_image" height="200px" align="middle">


### 1-4-1. 最適な遺伝子型の考案

正確に遺伝子型から形質値を予測するモデルを構築することが出来れば、目的に応じた遺伝子型を推定することが出来ます。

目的に応じてモデルを構築することで、
* 高収量の系統
* 病気や低/高温ストレスに強い系統
* 低肥料で効率よく収量が得られる系統

等、様々な目的に最適な遺伝子型を推定することが出来ます。

特に、構築したGenomic Predictionモデルが解釈しやすければ、その情報に基づいて推定することが可能です。
  * `形質値 = 遺伝子A × 2000 + 遺伝子 B × -1800`みたいな関係性が分かれば、遺伝子Aを持っていて、遺伝子Bに変異が入っている遺伝子型が望ましい、のような形ですね。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_ideal.png?raw=true" alt="GP_ideal" height="180px" align="middle">

### 1-4-2. Genomic Selection
　予測モデルの活用例として最も代表的なものが、最初に話したGenomic Selectionになります。

分野によってはGenomic SelectionとGenomic Predictionを同じものとして扱っているところもあります。

　先ほど、育成にコストや時間のかかる畜産の世界では、生まれてきた子供がどの程度遺伝的な能力を持っているか評価するために用いられているという話をしました。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/genomic_selection.png?raw=true" alt="genomic_selection" height="360px" align="middle">

そのため、これまではゲノム育種は特に動物の育種の世界で非常に重要なアプローチの１つとなっており、多くのGenomic Predictionモデルの開発は動物の研究から始まっていたりします。

　それでは、植物の方ではどうでしょうか。

植物育種の世界だと、動物と比べてより自由に親系統の交配が可能なので、

どんな組み合わせで掛け合わせると最適な系統を作ることが出来るのか予測することが出来ると育種に大きなメリットがあります。

「確率は低くても、最も収量が増える系統を作りたい」とか、「小規模の交配集団で収量を少し増加させる系統を作りたい。」など。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/GP_GS.png?raw=true" alt="GP_GS" height="400px" align="middle">

　植物の世界においても、従来の育種では収量の高い系統同士をかけあわせる等、形質値に基づいて交配組み合わせを考えていましたが、予測モデルに基づいて様々な組み合わせでどのような系統を作ることが出来るのか考えることで、より最適な組み合わせが見つかる可能性があります。

また、植物を対象としたゲノム育種と、動物を対象としたゲノム育種には栽培状況・生育環境など異なっていることがたくさんあるため、出来ることや目的に大きな違いがあります。



## (参考)1-3. 植物と動物の違い

Genomic Selectionは動物でも植物でも活用される選抜育種の方法の１つですが、

動物(畜産)を対象としたGenomic Prediction(Selection)と植物(作物)を対象としたGenomic Prediction(Selection)には様々な違いがあります。

| |  動物  |  植物  |
| ---- | ---- | ---- |
| 交配 | ヘテロが残る、時間がかかる | (自殖作物なら)固定化可能、Speed breedingが可能 |
|集団サイズ| 小さい | 多様、基本的に大きい |
|環境の影響| 形質による | 形質による |
|遺伝的変異| 意図的に入れるのは難しい | 交配や実験的に容易に入れられる |
|共有や保存| 共有や保存は難しい | 種として共有・保存が容易 |
| 価格 |  1個体あたり大  |  1個体あたり小  |
| GSによる早期選抜 | 利益が大きい | 利点はそれほど |
| 投資リターン |  投資に対するリターンが大きい  |  投資に対するリターンが小さい  |

　当然ながら実験的に扱いやすいのは植物です。多様な遺伝的変異を持つ集団を作ったり、理想的な遺伝子型を構築した際に、それを実際に作出するのは植物の方が容易なため、良いGenomic Predictionモデルを構築したり、応用するのも容易になります。

　一方、動物の場合は大きな集団サイズを用意するのが難しかったり、形質の調査にも時間がかかる等、難しい点があります。しかし、上手く選抜や改良を進めることが出来れば経済的な視点で見ると大きなリターンがあるというのも動物の特徴です。

　研究的な側面から言うと、植物は「上手くいかなければサンプルサイズを増やす」というアプローチを比較的簡単に行えるので、より実現可能性の高い、育種への応用に近い所まで研究を進められる可能性が高い傾向があります。動物はそう簡単にデータセットを増やせないので「より最適な統計手法を開発する」という方向に研究が発展し、様々な統計手法等が開発されやすい特徴があります。

(参考)
* [Genomic prediction unifies animal and plant breeding
programs to form platforms for biological discovery](https://www.nature.com/articles/ng.3920) (Hickey et al., 2017, Nature genetics)
* [Enhancing Genetic Gain through Genomic
Selection: From Livestock to Plants](https://doi.org/10.1016/j.xplc.2019.100005) (Xu et al., 2019, Plant Communications)

# 2. Genomic Predictionの解析手順・必要な知識等

実際に解析の流れを説明しながら体験していきましょう。今回はイネを対象としたGenomic Predictionを考えてみます。

大まかな流れとしては以下の様になります。
1. データの準備(集団の準備、Genotyping、Phenotyping)
2. 予測モデルの構築
3. 構築したモデルの応用

## 2-1. データの準備

　上述してある通り、良いGenomic Predictionモデルを構築するためには、大規模なデータが必要になります。

また、ただ単純に色んな系統をかき集めてくるのではなく、計画的に交配を行うことで、良いモデルを作りやすい集団を準備することもできます。

　ここではGWASやGenomic Predictionに活用しやすい様にデザインされた育種集団をいくつか紹介します。

　例えばRIL集団という集団はその代表的な例の一つです。RIL集団は、親系統からF2集団を作り、自家交配を繰り返し全ゲノムがホモ接合型になるまで世代を進めた集団になります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/GP_RIL.png?raw=true" alt="GP_RIL" height="320px">

RIL集団は親系統の遺伝子型がランダムに混ざってホモ接合型で固定されているため、ヘテロ接合の影響(雑種強勢など)を考慮する必要が無く、GWASなどを用いたQTLの探索にも非常に強力な材料になります。

　他にもMAGIC(Multi-parent Advanced Generation Inter-Cross)集団と呼ばれる複数品種からなる多系交雑集団などもあったりします。

MAGIC集団は複数の品種の遺伝子型がホモ接合でランダムに混ざっている様な集団です。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/GP_magic.png?raw=true" alt="GP_magic" height="320px">

figure: from [Bandillo et al., 2013](https://doi.org/10.1186/1939-8433-6-11)

今回の実習では、NAM(Nested Association Mapping)集団という交配集団のデータを使用します。

NAM集団は、片親の系統を固定し、もう片親に様々な系統を使用してRIL集団を作成したものになります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/GP_NAM.png?raw=true" alt="GP_NAM" height="320px">

この様な集団を作成することで、ある1つの系統に、様々な系統の遺伝子型がランダムに混ざった様な集団が作成できます。

この図で言うと「ひとめぼれ」系統に様々な系統の遺伝子型が混ざった様な集団になります。

準備には時間とコストがかかりますが、ひとめぼれ系統の収量を増加させたり、改良を進める際に有用な遺伝資源を探索するのには非常に強力な材料となります。

例えばひとめぼれ系統のどのゲノム領域をどんな系統の遺伝子型に入れ替えれば収量が上昇するか？といったことを探索するには最適な集団です。

今回はこのNAM集団の遺伝子型データと形質値データを使ってGenomic Predictionを行っていきます。

## 2-2. 予測モデルの構築

材料となる集団を作成し、形質値の調査(Phenotyping)や遺伝子型の調査(Genotyping)を終え、次はそれらのデータを用いて予測モデルを構築していきます。

「予測モデル」と言ってもピンとこないかもしれないので、極簡単な例で説明します。

### 2-2-1. 簡単な回帰モデルによる予測モデル

例えば、ある１箇所の遺伝子座の遺伝子型によって形質値が決まっている形質があったとします。

この時、遺伝子座の遺伝子型毎に形質を予測したい場合は以下の様に回帰式から予測することが出来ます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2025/images/GP_sample_model.png?raw=true" alt="GP_sample_model" height="320px">

着目した遺伝子座の遺伝子型を調べて、AAなのかATなのかTTなのかが分かれば、予測式からy(形質値)を計算して予測できる、という訳ですね。

簡単なサンプルデータで例を確認してみましょう。

遺伝子型(Genotype)はAA:0, AT:1, TT:2に変換してあります。

In [ ]:
# sample dataを見る
import pandas as pd
import numpy as np
sample_data = pd.read_csv("GP_linear_data.csv")
sample_data

このような15個体の形質値とあるSNPの遺伝子型データがあったとします。

形質値をSNPの遺伝子型から予測する回帰式($y=S\beta + \epsilon$)をこのデータから作ってみましょう。

Pythonには`scikit-learn`というライブラリがあり、回帰分析や機械学習など様々な解析を行うことが出来ます。

使い方は，前回の混合モデルと同様に、[公式のリファレンス](https://scikit-learn.org/stable/index.html)を見てみるとサンプルコード等が載っています。

それに加えて，例えば「回帰分析 Scikit-learn」のようなキーワードで検索すればたくさんの使用例が見つかります

In [ ]:
# 線形回帰を行い計算された回帰式を表示
from sklearn.linear_model import LinearRegression

geno = pd.DataFrame(sample_data.Genotype)
pheno = pd.DataFrame(sample_data.Phenotype)
model = LinearRegression()
model.fit(geno, pheno)
preds = model.predict(geno)
print(f"y = S×{model.coef_[0][0]:.3f}+{model.intercept_[0]:.3f}")

# グラフを表示
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.scatter(geno, pheno)
plt.plot(geno, preds, color = '#000000', linewidth=2)
plt.xticks([0, 1, 2],  ['AA', 'AT', 'TT'])
plt.show()

ではこの作成した回帰式 $y = S×23.436+50.941$ で形質をいくつか予測してみましょう。

下のコードの遺伝子型を0, 1, 2と変えてみて、式から予測される形質値と実際の形質値をいくつか比較してみましょう。

In [ ]:
#予測したい遺伝子型を入力
SNP = 1

#予測結果を表示
print("予測結果は", SNP * 23.436 + 50.941)

#入力した遺伝子型の実際の形質データを表示
print(f"\n測定データを見ると、SNPのGenotypeが{SNP}なのは…")
display(sample_data[(sample_data["Genotype"] == SNP)])

形質値が遺伝子型がAA(0)の時は50.941、AB(1)の時は74.377、BB(2)の時は97.813になると予測できる式が出来た訳ですね。

この時、作成した予測モデル(y = S×23.436+50.941)がどれくらい正しいか(モデルの予測精度)を確認してみます。

予測精度の指標としては、実際の形質値(実測値)と、モデル式から計算された形質の予測値を比較し、相関係数を求めることが多いです。

相関係数は0~1の範囲をとり、1に近いほど、予測値と実測値が相関していることを示す指標になります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/GP_accuracy.png?raw=true" alt="GP_accuracy" height="300px">


In [ ]:
# 予測値
predicted_values = model.predict(geno)
sample_data["Predicted"] = predicted_values
display(sample_data)

# 相関係数を計算して表示
print("\n相関係数: {}\n".format(np.corrcoef(sample_data["Phenotype"], sample_data["Predicted"])[0, 1]))

# 実測値と予測値の散布図
plt.scatter(sample_data["Phenotype"], predicted_values)
plt.xlabel("Observed phenotype values")
plt.ylabel("Predicted phenotype values")
plt.show()

### (参考)線形回帰で得られる直線

<small>※ややこしい話なので読み飛ばしてOKな場所です。</small>

　上のコードでは、$y = S×23.436+50.941$という式がPythonのライブラリによって計算されました。

回帰モデルによって得られる直線はどのような直線かというと、

　得られたデータについて、実測値と予測値(回帰直線上の点)のずれ（残差）が小さければデータの説明変数(今回だと遺伝子型)と目的変数(形質値)の関係を"うまく"表現している直線と言えるでしょう。

　今回おこなった単回帰モデルでは、残差の二乗の合計値、**残差平方和 (residual sum of squares)** が最も小さくなる係数 $\beta$ と誤差 $\epsilon$ の直線を求めています。

<img src="https://lh3.googleusercontent.com/pw/ACtC-3eIFmh8PDRx64eFArwdgxO2CGt3PEi272ny1dyqAMue0un_yL_GMgZ0CsyvBnX4lEC9BfOEdfTNGsiEG-R4xZDPM9zMHwHcINcnQFxcdTmSgsF7LotLsBpwzs0S49fZtN1fQrbHY7JrB9m2kwuDGb9r=w815-h560-no?authuser=0" alt="least squares" height="180px">

$$ 残差平方和: \sum_{i=1}^{N} (\hat{y}_{i} - y_i)^2 = \sum_{i=1}^{N} (S_i\beta + \epsilon - y_i)^2 $$

  - $S_{i}$: i番目のデータの遺伝子型の値
  - $y_{i}$: i番目のデータの形質値
  - $\hat{y}_{i}$: i番目のデータの形質値の予測値(回帰直線上の値)
  - $N=1,2,3,...,n$: データの個数

　残差平方和のように、最小化を目指す関数のことを、**目的関数 objective function** や **コスト関数 cost function** と呼びます。  

　殆どの機械学習の各アルゴリズムは、何らかの目的関数を持っています。データに基づいて、目的関数の最小化（ときには最大化）をおこなってパラメータの値を求めていきます(今回だと$\beta$や$\epsilon$)。どのような目的関数を選択するかも良いモデルを作成するためには非常に重要です。

<small>※ 線形回帰モデルでパラメータ（係数や誤差）の最適値を求める方法は、「係数$\beta$ と 誤差$e$ の最適値をどうやって求めているか？」を参照してください。</small>






#### 係数$\beta$ と 誤差$\epsilon$ の最適値をどうやって求めているか？

　scikit-learnの `LinearRegression()` は、**最小二乗法 least squares method**と呼ばれる方法を使って、残差平方和が最小値（極小値）になる係数$\beta$ と誤差$\epsilon$ を求めています。最小二乗法の概要は次のとおりです。

1. 目的関数（残差平方和）
$$ \sum_{i=1}^{N} (\hat{y}_{i} - y_i)^2 = \sum_{i=1}^{N} (S_i\beta + \epsilon - y_i)^2 $$
  - $S_{i}$: i番目のデータの説明変数の値
  - $y_{i}$: i番目のデータの目的変数の値（実測値）
  - $\hat{y}_{i}$: i番目のデータの目的変数の値（予測値）
  - $N=1,2,3,...,n$: データの個数
  
1. 目的関数の$\sum$の部分を展開すると、
$$ \sum_{i=1}^{N} (\beta S_i + \epsilon - y_i)^2 = \beta^{2}\sum_{i=1}^{N}{S_{i}^{2}} + N\epsilon^2 + \sum_{i=1}^{N}{y_{i}^{2}} - 2\beta\sum_{i=1}^{N}{S_{i}y_{i}} + 2\beta \epsilon\sum_{i=1}^{N}{S_{i}} -2\epsilon\sum_{i=1}^{N}{y_{i}} $$

1. 上の展開式を$\beta$の関数として考えると、下に凸な二次関数であることがわかります。
$$ \sum_{i=1}^{N} (\beta S_i + \epsilon - y_i)^2 = \Bigl(\sum_{i=1}^{N}{S_{i}^{2}} \Bigr) \beta^{2} - \Bigl( 2\sum_{i=1}^{N}{S_{i}y_{i}} + 2 \epsilon\sum_{i=1}^{N}{S_{i}} \Bigr) \beta + \Bigl( N\epsilon^2 + \sum_{i=1}^{N}{y_{i}^{2}}　-2\epsilon\sum_{i=1}^{N}{y_{i}} \Bigr) $$
同様に、$\epsilon$の関数として考えた場合も下に凸な二次関数となります。したがって、残差平方和が最小となる$\beta$と$\epsilon$は、両者の関数が同時に極小値となるときの値と一致しそうです。

1. そこで、$\beta$や$\epsilon$についての偏微分関数が0になる値（極小値）を調べます。
 $$ (\beta について偏微分)=0 $$
 $$ (\epsilon について偏微分)=0 $$
1. 上の2つの方程式を解くと、目的関数が極小値となる$\beta$や$e$の値が求まります


<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/least_squares_method.png?raw=true" alt="least squares method" height="180px">


### 2-2-2. 重回帰式

さて、先ほどの例では、たった１つの遺伝子座の遺伝子型によって形質が決まっている場合でした。

しかし基本的に、たった１つの遺伝子で決定される形質というのはあまりありません。大抵複数の遺伝子が関与しています。

そこで次は、少し増やして３箇所の遺伝子座の遺伝子型によって形質値が決まっている形質を考えてみましょう。

この時には、先ほどの単回帰分析とは異なり、複数の説明変数を回帰式に入れた重回帰分析を行います。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/GP_sample_model2.png?raw=true" alt="GP_sample_model2" height="200px">

簡単なサンプルデータで例を確認してみましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2025/images/GP_sample_model2_2.png?raw=true" alt="GP_sample_model2" height="200px">

In [ ]:
# sample data
sample_data = pd.read_csv("GP_linear_data2.csv")
sample_data

まずは先ほど同様、SNP1だけで形質を予測しようとしてみましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2025/images/GP_sample_model2_3.png?raw=true" alt="GP_sample_model2" height="200px">

In [ ]:
# 単回帰分析
model = LinearRegression()
# SNP1の情報だけ取得
SNP1_genotype = sample_data.iloc[:, :1]
# 形質データ
phenotype = sample_data.iloc[:, 3]
# 単回帰式を作成
model = model.fit(SNP1_genotype, phenotype)
print(f"y = SNP1×{model.coef_[0]:.3f}+{model.intercept_:.3f}")

得られた式を元に、予測結果と実際の形質値を比較してみましょう。

In [ ]:
#予測したい遺伝子型を入力
SNP1 = 0

#予測結果を表示
print("予測結果は", SNP1 * 30.258 + 103.692)

#入力した遺伝子型の実際の形質データを表示
print(f"\n測定データを見ると、SNP1のGenotypeが{SNP1}なのは…")
display(sample_data[(sample_data["SNP1_Genotype"] == SNP1)].iloc[:, [0, 3]])

予測値と実測値の間の相関係数を計算してみると…

In [ ]:
# 実測値と予測値の散布図
# 予測値を計算
y_preds = model.predict(sample_data.iloc[:, :1])
# 散布図を描写
plt.scatter(sample_data.iloc[:, 3], y_preds)
plt.xlabel("Observed phenotype values")
plt.ylabel("Predicted phenotype values")
# 相関係数を計算して表示
print("相関係数: {}".format(np.corrcoef(sample_data["Phenotype"], y_preds)[0, 1]))

相関係数も低く、１つのSNPの遺伝子型だけからでは形質値を上手く予測できないことが分かりました。

では次にSNP3つ全て使って予測式を作るために、重回帰分析を行ってみます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2025/images/GP_sample_model2_4.png?raw=true" alt="GP_sample_model2" height="200px">

In [ ]:
# 重回帰分析
model = LinearRegression()
# 全SNPの情報を取得
SNP_genotypes = sample_data.iloc[:, :3]
# 形質データ
phenotype = sample_data.iloc[:, 3]
# 単回帰式を作成
model = model.fit(SNP_genotypes, phenotype)
print(f"y = SNP1×{model.coef_[0]:.3f}+SNP2×{model.coef_[1]:.3f}+SNP3×{model.coef_[2]:.3f}+{model.intercept_:.3f}")

このように、重回帰分析を行うと、3か所のSNPの遺伝子型から、形質値が予測できる式が出来ました。

$$ y = SNP1×23.380+SNP2×48.530+SNP3×10.459+52.056 $$

ではこの作成した回帰式で形質をいくつか予測してみましょう。

下のコードの遺伝子型を色々変えてみて、式から予測される形質値と上の表の実際の形質値をいくつか比較してみましょう。

In [ ]:
#予測したい遺伝子型を入力
SNP1 = 2
SNP2 = 1
SNP3 = 1

#予測結果を表示
print("予測結果は", SNP1 * 23.380 + SNP2 * 48.530 + SNP3 * 10.459 + 52.056)

#入力した遺伝子型の実際の形質データを表示
print(f"\n測定データを見ると、SNP1が{SNP1},SNP2が{SNP2},SNP3が{SNP3}なのは…")
display(sample_data[(sample_data["SNP1_Genotype"] == SNP1) & (sample_data["SNP2_Genotype"] == SNP2) & (sample_data["SNP3_Genotype"] == SNP3)])


実際の形質値と作成した回帰式で予測した形質値がどれくらい近いのかをグラフにして、回帰モデルの予測精度を見てみます。

In [ ]:
# 実測値と予測値の散布図
# 予測値を計算
y_preds = model.predict(sample_data.iloc[:, :3])
# 散布図を描写
plt.scatter(sample_data.iloc[:, 3], y_preds)
plt.xlabel("Observed phenotype values")
plt.ylabel("Predicted phenotype values")
# 相関係数を計算して表示
print("相関係数: {}".format(np.corrcoef(sample_data["Phenotype"], y_preds)[0, 1]))

　この様に、1つのSNPだけでは説明出来なかった形質も、3つのSNPの情報からだと上手く予測出来ました。

今回のサンプルデータでは上手く形質値と遺伝子型の関係性を表すことが出来たことになります。

このように、回帰式の様な形で、形質値と遺伝子型の関係性を表す良い予測モデルを作ることが出来れば、

遺伝子型の情報を調べるだけでどんな形質値になるか予測することが出来る様になるわけです。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/GP_image.png?raw=true" alt="GP_image" height="180px" align="middle">

<br>
<br>

しかし、より多数の複雑な遺伝子に支配されている場合には、

遺伝子の相互作用など、今回扱った回帰式では形質値と遺伝子型の関係性を説明しきれない場合があったり、

形質がどの遺伝子によってコントロールされているかは殆ど分かっていないので、

予測モデルにどの場所の遺伝子型の情報を入れるべきかを決めるのがとても難しかったりします。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/GP_unknown.png?raw=true" alt="GP_unknown" height="250px" align="middle">

### 2-2-3. NAM集団のデータで回帰モデル

では実際にNAM集団のデータを元に回帰モデルで予測式を作ってみましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/GP_NAM.png?raw=true" alt="GP_NAM" height="320px">

今回は747系統のNAM集団の形質情報(１つの穂につく籾数)と6366 SNPの遺伝子型情報を用います。

うまく籾数を予測するモデルを作ることが出来れば、収量を上げるために籾数を増やすにはどんな遺伝子型が良いか分かるようになる訳です。

GenotypeデータとPhenotypeデータを下のコードから確認出来ます。

In [ ]:
# Genotype data
genotype = pd.read_csv("GP_sample_genotype.csv")
genotype

In [ ]:
# Phenotype data
phenotype = pd.read_csv("GP_sample_phenotype.csv")
phenotype

まずはこれら6366個のSNPの遺伝子型全ての情報を使って重回帰モデルを作り、籾数を予測するモデルを構築してみましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/GP_sample_model3.png?raw=true" alt="GP_sample_model3" height="80px">

In [ ]:
# 線形回帰
model = LinearRegression()
model = model.fit(genotype.iloc[:, 2:].T, phenotype["GN_mean"])
equation = "y = "
for i, j in enumerate(model.coef_):
    equation += "SNP{}×{} + ".format(str(i+1), str(j))
equation += str(model.intercept_)
equation

In [ ]:
# 予測値
y_preds = model.predict(genotype.iloc[:, 2:].T)
phenotype["Predicted"] = y_preds
display(phenotype)

In [ ]:
# 予測精度
plt.scatter(phenotype["GN_mean"], y_preds)
plt.xlabel("Observed phenotype values")
plt.ylabel("Predicted phenotype values")
print("相関係数: {}".format(np.corrcoef(phenotype["GN_mean"], y_preds)[0, 1]))

## 2-3. 構築したモデルの応用

相関係数が0.99と非常に高い精度の予測モデルが出来たかと思います。

それでは重回帰モデルを使用して構築したこの予測モデルを、育種に活用していきましょう。

...と行きたいところですが、実はこの予測モデル、一見凄く予測精度が高いモデルに見えますが、様々な問題点が残っています。

(今回の課題はどんな問題があるか考えてみる所になります。次回解説します。)

というわけで、今日はこちらで構築した問題の少ないGenomic Predictionモデルを用意したので、それを使って予測モデルの活用例をいくつか試してみましょう。



# 3. Genomic Predictionモデルの応用を体験してみる

今回は、NAM集団のデータを元に、籾数を予測する予測モデルの構築を試みました。

もし良い予測モデルが出来れば、籾数を増加させるにはどんな遺伝子型が良いか分かる、という話でしたが、

まず下のコードで、NAM集団のデータから作成した良い予測モデルを表示させられます。

In [ ]:
# 予測モデルの構築
import warnings
warnings.filterwarnings('ignore')
from genomic_prediction import split_dataset
from genomic_prediction import make_genomic_prediction_model
from genomic_prediction import check_equation
test_genotype, test_phenotype, train_genotype, train_phenotype = split_dataset(genotype, phenotype, "GN_mean", test=0.2)
GN_prediction_model = make_genomic_prediction_model(train_genotype, train_phenotype, "GN_mean")
print("\nPrediction model is ...")
print(check_equation("GN_mean", GN_prediction_model), "\n")

$籾数 = 113.72432825279989 + SNP4×0.6886779521104645 + SNP7×-0.05215675945160048 + ... $

今回はこの予測モデルを利用して以下の2点をやってみましょう。

* 理想的な遺伝子型を探してみる
* 最適な交配組み合わせを探してみる

## 3-1. 理想的な遺伝子型を探してみる

予測モデルが出来た、ということは遺伝子型を入力すればどんな形質値になるかが分かる、ということになります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/GP_ideal.png?raw=true" alt="GP_ideal" height="150px" align="middle">

つまり、モデルに予測させて高い籾数になる遺伝子型を見つけ出すことが出来れば良い訳です。

下のコードを動かすと任意のゲノム領域の遺伝子型がひとめぼれ系統からREXMONTという系統に変わった時に、どんな形質値になるかを計算してくれます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2025/images/GP_genotype_change.png?raw=true" alt="GP_genotype_change" height="250px" align="middle">


変化させる領域を色々変えて籾数が最も大きくなるような遺伝子型を試してみましょう。

---

`regions = [ [領域1], [領域2], ... [領域x] ]`という形で複数の領域を指定することもできます。

例えば、`[['chr01', 12000000, 20000000], ['chr05', 0, 5000000], ['chr12', 10000000, 15000000]]`と書くと、

* chr01の12000000~20000000bpの領域
* chr05の0~5000000bpの領域
* chr12の10000000~15000000bpの領域

がREXMONTに変わった時の値を計算してくれます。

' 'や , の付け忘れ、chr1ではなくchr01、最初と最後の[]の付け忘れなどに注意して色々試してみてください。

※各染色体の長さは長いもので40000000bpくらい、短いと20000000bpくらいです。


In [ ]:
# 任意の場所の遺伝子型が別の系統に変わるとどんな形質が出来るか
from genomic_prediction import predict_customized_genotype
regions = [['chr01', 10000000, 20000000], ['chr03', 10000000, 20000000]]
predict_customized_genotype(genotype, regions, GN_prediction_model)

## 3-2. 最適な交配組み合わせを探してみる

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/GP_GS.png?raw=true" alt="GP_GS" height="300px" align="middle">

NAM_001 ~ NAM_747までの747系統の中でどれとどれを交配させるとより良い系統を作ることが出来るでしょうか？

下のコードを動かすと任意の2系統を交配し、F2集団を作った時にどんな形質値の集団になるかを計算してくれます。

`NAM_001`と`NAM_002`の部分を色々変えて試してみましょう。

In [ ]:
# 組み合わせでどんな形質が出来るか
from genomic_prediction import predict_progeny_phenotype
predict_progeny_phenotype("NAM_001", "NAM_002", 100, genotype, GN_prediction_model)

# まとめ

　今回、Genomic Predictionの概要と解析の流れ、応用例をみてきました。
- Genomic Prediction:
  - 「ある遺伝子型を持つ生物がどんな形質値になるのか」を予測するアプローチ
  - 理想的な遺伝子型や最適な交配組み合わせを決めるのに役立つ

次回からは予測モデル構築に関してより具体的な進め方や様々な統計モデル・手法の紹介などを学んでいきます。

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>